# 最終的なモデル詰め合わせ

In [76]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# kNN
from sklearn.neighbors import KNeighborsClassifier

In [6]:
# Path
dbPath = "db_list.csv"
queryPath = "query_list.csv"

In [11]:
# read data
db_df = pd.read_csv(dbPath, index_col=0)
query_df = pd.read_csv(queryPath, index_col=0)

In [12]:
db_df.head()

,dct_0,dct_1,dct_2,dct_3,dct_4,dct_5,dct_6,dct_7,dct_8,dct_9,...,dct_216,dct_217,dct_218,dct_219,dct_220,dct_221,dct_222,dct_223,dct_224,target
0,0.812763,0.272423,0.177267,0.262133,0.230776,0.255399,0.189310,0.279024,0.155362,0.268883,...,0.228107,0.209880,0.233954,0.249645,0.227619,0.223897,0.211591,0.215112,0.234028,0.0
1,0.824317,0.285164,0.189974,0.263251,0.235999,0.265722,0.199164,0.279436,0.176147,0.262119,...,0.220713,0.216296,0.215499,0.244427,0.238536,0.208056,0.222545,0.221152,0.220127,0.0
2,0.826549,0.227953,0.120643,0.218246,0.210585,0.235652,0.162152,0.232881,0.130341,0.223699,...,0.223130,0.223681,0.236513,0.228123,0.201495,0.224386,0.223604,0.231683,0.231720,0.0
3,0.806723,0.251155,0.105308,0.217036,0.183272,0.209225,0.151727,0.248162,0.129832,0.241668,...,0.225695,0.230213,0.249365,0.230700,0.204350,0.224435,0.230503,0.218306,0.228928,0.0
4,0.858456,0.247600,0.184433,0.247196,0.241287,0.259901,0.171110,0.243555,0.149487,0.252953,...,0.238263,0.211677,0.232359,0.236703,0.224809,0.216576,0.234454,0.215928,0.244901,0.0


In [13]:
query_df.head()

,dct_0,dct_1,dct_2,dct_3,dct_4,dct_5,dct_6,dct_7,dct_8,dct_9,...,dct_216,dct_217,dct_218,dct_219,dct_220,dct_221,dct_222,dct_223,dct_224,target
0,0.904674,0.194109,0.130995,0.174769,0.195891,0.177870,0.152082,0.194753,0.131312,0.205085,...,0.244702,0.224199,0.213333,0.223402,0.210794,0.220879,0.234462,0.235203,0.224115,0.0
1,0.784270,0.224606,0.139211,0.248058,0.153540,0.216527,0.126956,0.223850,0.126157,0.192243,...,0.239503,0.236071,0.233055,0.233057,0.221392,0.190645,0.216429,0.212288,0.208633,0.0
2,0.876182,0.195360,0.210239,0.135471,0.247396,0.205266,0.253241,0.195721,0.188476,0.205977,...,0.229111,0.236003,0.231136,0.236494,0.230394,0.212862,0.238171,0.210575,0.218420,0.0
3,0.691964,0.134280,0.224722,0.162955,0.213020,0.245184,0.141536,0.234308,0.111738,0.229067,...,0.220245,0.221221,0.233103,0.200231,0.245400,0.235332,0.238871,0.197243,0.242605,0.0
4,0.791492,0.153364,0.171210,0.174341,0.203113,0.191042,0.189231,0.192029,0.169168,0.157802,...,0.222803,0.178768,0.241913,0.217591,0.251577,0.210878,0.220035,0.237752,0.204233,0.0


In [80]:
db_featture_df = db_df.drop(["target"], axis=1).copy()
db_target_df = db_df["target"].copy()
query_featture_df =query_df.drop(["target"], axis=1).copy()
query_target_df = query_df["target"].copy()

# 単純マッチング

In [66]:
result = []
test_min = []
prediction_df = pd.DataFrame(query_target_df)
for i in range(query_featture_df.shape[0]):
    minimum_id = (db_featture_df - query_featture_df.iloc[i]).abs().sum(axis=1).idxmin()
    test_min.append((db_featture_df - query_featture_df.iloc[i]).abs().sum(axis=1))
    prediction_df.loc[i, "predict"] = db_target_df[minimum_id]

In [67]:
prediction_df.head()

,target,predict
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,8.0
4,0.0,0.0


In [68]:
correct_num = (prediction_df["target"] == prediction_df["predict"]).sum()
accuracy = correct_num / prediction_df.shape[0]

In [69]:
accuracy

0.7758620689655172

# kNN

In [97]:
DIV_NUM = 3 # k
DIST_SETTING = 2 # ユークリッド=2, マンハッタン=1

In [98]:
knn = KNeighborsClassifier(n_neighbors=DIV_NUM, p=DIST_SETTING, metric="minkowski")
knn.fit(db_featture_df.values, db_target_df.values)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [99]:
prediction = knn.predict(query_featture_df.values)
prediction_df = pd.DataFrame(query_target_df)
prediction_df["predict"] = prediction

In [100]:
correct_num = (prediction_df["target"] == prediction_df["predict"]).sum()
accuracy = correct_num / prediction_df.shape[0]

In [101]:
accuracy

0.7068965517241379

# 